# データの確認  
途中くらいまでデータ読み込みのコードになります。  
このKernelでは、  
・画像の確認  
・マスク画像の確認  
・悪性度の高い組織と正常組織の比較  
・データの分布  
  
などについて調べます。  

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

# There are two ways to load the data from the PANDA dataset:
# Option 1: Load images using openslide
import openslide
# Option 2: Load images using skimage (requires that tifffile is installed)
import skimage.io
import random
import seaborn as sns
import cv2

# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import PIL
from IPython.display import Image, display

# Plotly for the interactive viewer (see last section)
import plotly.graph_objs as go

In [ ]:
# 大元のディレクトリ
BASE_PATH = '../input/prostate-cancer-grade-assessment'

# trainingとmask画像データのディレクトリ
data_dir = f'{BASE_PATH}/train_images'
mask_dir = f'{BASE_PATH}/train_label_masks'


# train.csv,その他testとsubmissionのディレクトリ
train = pd.read_csv(f'{BASE_PATH}/train.csv').set_index('image_id')
test = pd.read_csv(f'{BASE_PATH}/test.csv')
submission = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')

In [ ]:
display(train.head())
print("Shape of training data :", train.shape)
print("unique data provider :", len(train.data_provider.unique()))
print("unique isup_grade(target) :", len(train.isup_grade.unique()))
print("unique gleason_score :", len(train.gleason_score.unique()))

In [ ]:
#ISUPグレード5(悪性度の高い)の画像をピックアップ
res_5 = train.query("isup_grade == 5 and data_provider == 'radboud'")
res_5.head()

In [ ]:
#ISUPグレード0(正常組織)の画像をピックアップ
res_0 = train.query("isup_grade == 0 and data_provider == 'radboud'")
res_0.head()

In [ ]:
def display_image(image_id,x,y,level):
    isup_grade = train.loc[image_id]["isup_grade"]
    biopsy = openslide.OpenSlide(os.path.join(data_dir,image_id + '.tiff'))
    biopsy_mask = openslide.OpenSlide(os.path.join(mask_dir,image_id + '_mask.tiff'))
    print("画像情報")
    print("image_id",image_id)
    print("ISUPグレード：",isup_grade)
    print("画像サイズ:",biopsy.dimensions)
    zoom = ["高い","普通","低い"]
    print("拡大率:",zoom[level])
    
    region = biopsy.read_region((x,y), level, (512, 512))
    region_mask = biopsy_mask.read_region((x,y), level, (512, 512))
    
    f,ax = plt.subplots(1,2,figsize=(16,18))
    cmap = matplotlib.colors.ListedColormap(['black', 'gray', 'green', 'yellow', 'orange', 'red'])
    ax[0].imshow(region)
    ax[0].axis('off')
    ax[0].set_title(f"ID: {image_id}\nISUP: {isup_grade}")
    ax[1].imshow(np.asarray(region_mask)[:,:,0], cmap=cmap, interpolation='nearest', vmin=0, vmax=5)
    ax[1].axis('off')
    ax[1].set_title("mask_image")

    plt.show() 

In [ ]:
images = ["007433133235efc27a39f11df6940829", #grade0
          "01642d24ac5520681d6a20f6c42dc4fe", #grade0
          "02577ddcd838f2559936453b6071dc17", #grade0
          "00928370e2dfeb8a507667ef1d4efcbb", #grade5
          "00c15b23b30a5ba061358d9641118904", #grade5
          "00c52cb4db1c7a5811a8f070a910c038"  #grade5
         ]

# トレーニング画像の確認  
まず最初に、トレーニング画像と、そのどの場所が悪性であるのかをマスク画像を用いて確認していきます。  
正常組織と悪性度の高い組織について、様々な拡大率で2例ずつ見てみます。  


# マスク画像のグレードについて  
各マスク画像の色は、次のグレードを表しています。  
## 黒：背景。組織ではない部分  
## 灰色：結合組織や非上皮組織(癌には関係ない組織のことです)  
## 緑：正常な上皮組織  
## 黄色：がん組織(グリソン分類３)  
## オレンジ：がん組織(グリソン分類４)  
## 赤：がん組織(グリソン分類５)  

まず、正常な組織(ISUPグレード0)の画像について3つの倍率で見てみます  
画像から、以下のことが分かります。  
・多くの場所は診断に関係ない非上皮性組織である。  
・上皮性組織には細胞核が集中している。  
・上皮性組織は文字通り組織の表面に位置することが分かる。  
・核の配置が規則的である。  

In [ ]:
#一番遠い視野での画像
display_image("007433133235efc27a39f11df6940829",2000,8000,2)

In [ ]:
#同画像を拡大したもの
display_image("007433133235efc27a39f11df6940829",4000,11000,1)
#更に上皮組織を拡大したもの
display_image("007433133235efc27a39f11df6940829",4300,12500,0)

次に、悪性度の高い組織画像について、同じく3段階の倍率で見てみます。  
以下のことが分かります。  
・マスク画像において、悪性度の高い赤色を示す割合が非常に多い  
・組織表面でなくとも、細胞核の密集率が非常に高い  
・拡大率中の画像を正常細胞と比較した時に、とにかく細胞核の数が多いのが目立つ  
・核の形、大きさがまばらである  
・正常細胞に対して、核の配置が規則的でない  

In [ ]:
#遠くから
display_image("00928370e2dfeb8a507667ef1d4efcbb",2000,8000,2)


In [ ]:
#赤いマスクがされている部分について拡大
display_image("00928370e2dfeb8a507667ef1d4efcbb",5000,8000,1)
#更に拡大
display_image("00928370e2dfeb8a507667ef1d4efcbb",6200,8700,0)

In [ ]:
#正常細胞例2
display_image("01642d24ac5520681d6a20f6c42dc4fe",5000,5000,2)

In [ ]:
#緑マスク部にめがけて拡大。
display_image("01642d24ac5520681d6a20f6c42dc4fe",5000,7000,1)
#更に拡大
display_image("01642d24ac5520681d6a20f6c42dc4fe",6000,7300,0)

In [ ]:
#ISUPグレード5(悪性度高)細胞例2
display_image("00928370e2dfeb8a507667ef1d4efcbb",2500,8000,2)

In [ ]:
#悪性部に拡大
display_image("00928370e2dfeb8a507667ef1d4efcbb",4500,8500,1)
#さらに拡大
display_image("00928370e2dfeb8a507667ef1d4efcbb",5500,8700,0)

In [ ]:
#また、同画像のオレンジ(グリソンスコア4)の部分についても見てみる。
display_image("00928370e2dfeb8a507667ef1d4efcbb",4500,12000,1)
#さらに拡大
display_image("00928370e2dfeb8a507667ef1d4efcbb",5200,12800,0)

# データの分布  
画像の特徴を一通り見終わったので、次は主要なデータ分布を調べます。  
まずは研究所ごとのデータの提供割合について見てみます。

In [ ]:
#データを提供してる研究所は以下の2つ。
train["data_provider"].unique()

In [ ]:
def plot_ct(data):
    left = [i for i in range(len(data))]
    height = [i[1] for i in data]
    labels = [i[0] for i in data]
    plt.bar(left, height, width=0.5,linewidth=2, tick_label=labels)
    plt.title("data_provider")
    plt.ylabel("count")
    plt.xlabel("provider")
    plt.show()
    

data = [
    ['karolinska',(train["data_provider"] =='karolinska').sum()],
    ['radboud',(train["data_provider"] =='radboud').sum()]
]


plot_ct(data)
print("施設ごとのデータ提供割合：")
for i,j in data:
    print(i,":",round(j/len(train)*100,2),"%")

2施設のデータに偏りは無いことが分かりました。  
次に、ISUPグレードの分布を確認します。

In [ ]:
def plot_isup(data):
    left = [i for i in range(len(data))]
    height = [i[1] for i in data]
    labels = [i[0] for i in data]
    plt.bar(left, height, width=0.5,linewidth=2, tick_label=labels)
    plt.title("isup_grade")
    plt.ylabel("count")
    plt.xlabel("isup_grade")
    plt.show()
    

data = [[i,(train["isup_grade"] == i).sum()] for i in range(6)]

plot_isup(data)
print("それぞれのISUPグレードの割合：")
for i,j in data:
    print("グレード",i,":",round(j/len(train)*100,2),"%")

グレード0(陰性)とグレード1(悪性度の低いグレード)の割合が高く、他のデータの割合がほとんど同じであることが分かります。  
特にグレード3~5は12%に統一されており、わざとこのように仕組んでいるのかな？と感じます。  

テストケースでこのような分布になっているとは限らないので、注意が必要です。  
  
更に、施設ごとのISUPグレードも見ておきましょう。  

In [ ]:
print('karolinskaのみ')
data = [[i,((train["isup_grade"] == i) & (train["data_provider"] =='karolinska')).sum()] for i in range(6)]

plot_isup(data)
print("それぞれのISUPグレードの割合：")
for i,j in data:
    print("グレード",i,":",round(j/len(train)*100,2),"%")

In [ ]:
print('radboudのみ')
data = [[i,((train["isup_grade"] == i) & (train["data_provider"] =='radboud')).sum()] for i in range(6)]

plot_isup(data)
print("それぞれのISUPグレードの割合：")
for i,j in data:
    print("グレード",i,":",round(j/len(train)*100,2),"%")

施設ごとにかなりバラツキがあるのが分かりました。  
特徴としては、  
karolinskaはグレード0,1の割合が比較的高い。  
radboudは全体的に均一な分布となっている。  

テストケースにおいて、data_prividerは事前に特徴量として渡されるデータです。  
このようなデータの偏りがあることを考慮すると、施設ごとにモデルを作成する、という方法が有効な手段になりそうです。  

最後に、ISUPグレードではなく、グリソン分類についても分布を見ておきたいと思います。  
グリソン分類は、細胞上皮においてその面積が占める割合が大きいものから2つ撮ってきて、3+3～5+5までの分類で表します。(陰性の場合は0+0またはnegative)  
更にその和からISUPグレードを求める、というふうになっています。(コンペトップにISUP算出法の図があります)  
まずはデータ全体での分布を見てみましょう。  

In [ ]:
train["gleason_score"].unique()

In [ ]:
def plot_gleason(data):
    left = [i for i in range(len(data))]
    height = [i[1] for i in data]
    labels = ["nega" if i[0] == "negative" else i[0] for i in data]
    plt.bar(left, height, width=0.5,linewidth=2, tick_label=labels)
    plt.title("gleason_score")
    plt.ylabel("count")
    plt.xlabel("gleason_score")
    plt.show()
    

data = [[i,(train["gleason_score"] == i).sum()] for i in train["gleason_score"].unique()]
plot_gleason(data)
print("それぞれのグリソン分類の割合：")
for i,j in data:
    print(i,":",round(j/len(train)*100,2),"%")

negativeと0+0は同義ですが、どうやら研究所によって表記が異なるようです。  
0+0：karolinskaでの陰性表記  
negative：radboudでの陰性表記  

In [ ]:
for i in ["karolinska","radboud"]:
    res = train[train["data_provider"] == i]
    print("0+0 in ",i,":",any("0+0" == i for i in res["gleason_score"]))
    print("negative in ",i,":",any("negative" == i for i in res["gleason_score"]))


施設ごとのデータ分布も確認します。

In [ ]:
print('karolinskaのみ')

data = [[i,((train["gleason_score"] == i) & (train["data_provider"] =='karolinska') ).sum()] for i in train["gleason_score"].unique()]
plot_gleason(data)
print("それぞれのグリソン分類の割合：")
for i,j in data:
    print(i,":",round(j/len(train)*100,2),"%")

In [ ]:
print('radboudのみ')

data = [[i,((train["gleason_score"] == i) & (train["data_provider"] =='radboud') ).sum()] for i in train["gleason_score"].unique()]
plot_gleason(data)
print("それぞれのグリソン分類の割合：")
for i,j in data:
    print(i,":",round(j/len(train)*100,2),"%")

この2つのデータの比較からは、
karolinskaでは5+から始まる症例はほとんど扱っていない   
そのため、karolinskaにおけるISUPグレード5のほとんどは4+5のグリソン分類によるものである。  
karolinskaでは陽性の場合3+3か3+4が大半を占めている。  
radboudではデータの分布が比較的均一であるが、5+から始まるデータはやはり少ない。  

といったことが読み取れます。